<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/4_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import html, dcc, Input, Output, State
import time
import threading
import pandas as pd
import plotly.express as px
import numpy as np
import random

app = dash.Dash(__name__)
server = app.server

# 步驟清單
step_names = [
    "撈取資料",
    "確認與篩選資料",
    "資料預處理",
    "進行 Rule Check",
    "繪圖",
    "上拋資料"
]

# 儲存進度資訊
progress = {
    'current_index': -1,
    'step_times': [None] * len(step_names),
    'step_status': ['pending'] * len(step_names),  # pending / success / error
    'fig': None,
    'error': False
}

# 模擬報告流程
def run_report_process():
    df = None
    start_times = []

    for i in range(len(step_names)):
        if progress['error']:
            break  # 出錯就不要繼續

        progress['current_index'] = i
        progress['step_status'][i] = 'running'
        step_start = time.time()

        time.sleep(1)  # 模擬執行

        # 模擬在第4步有10%機率失敗
        if i == 3 and random.random() < 0.1:
            progress['step_status'][i] = 'error'
            progress['error'] = True
            break

        # 特別在畫圖步驟產生圖表
        if i == 4:
            df = pd.DataFrame({
                'player_id': range(1, 101),
                'profit': np.random.normal(1000, 300, 100),
                'game_type': np.random.choice(['A', 'B', 'Cash', 'MTT'], 100)
            })
            fig = px.histogram(df, x='profit', color='game_type', nbins=20, title="玩家盈利分佈")
            progress['fig'] = fig

        step_end = time.time()
        progress['step_times'][i] = round(step_end - step_start, 2)
        progress['step_status'][i] = 'success'

app.layout = html.Div([
    html.H2("報告產出流程 - 每步時間＋錯誤偵測＋顏色控制"),
    html.Button("開始產出報告", id="run-btn", n_clicks=0),
    html.Div(id="step-list", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Div(id="progress-label", style={"marginTop": "20px", "fontSize": "16px"}),
    html.Progress(id="progress-bar", value=0, max=len(step_names),
                  style={"width": "100%", "height": "30px", "marginBottom": "20px"}),
    dcc.Interval(id="interval", interval=1000, n_intervals=0, disabled=True),
    dcc.Loading(html.Div(id="chart-output")),
    html.Div(id="to-comment-div")  # 新增一個區域顯示to_comment按鈕
])

@app.callback(
    Output("interval", "disabled"),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_report(n_clicks):
    # 初始化
    progress['current_index'] = -1
    progress['step_times'] = [None] * len(step_names)
    progress['step_status'] = ['pending'] * len(step_names)
    progress['fig'] = None
    progress['error'] = False

    # 開啟多執行緒執行報告
    thread = threading.Thread(target=run_report_process)
    thread.start()

    return False  # 這裡保持 Interval 開啟

@app.callback(
    Output("step-list", "children"),
    Output("progress-bar", "value"),
    Output("progress-label", "children"),
    Output("chart-output", "children"),
    Output("to-comment-div", "children"),  # 更新 to-comment 按鈕的顯示
    Output("interval", "disabled", allow_duplicates=True),  # 停止 interval
    Input("interval", "n_intervals"),
    prevent_initial_call=True
)
def update_ui(n):
    current = progress['current_index']
    step_status = progress['step_status']
    step_times = progress['step_times']
    steps_total = len(step_names)
    chart = dcc.Graph(figure=progress['fig']) if progress['fig'] else ""

    step_elements = []
    for i, step in enumerate(step_names):
        status = step_status[i]

        if status == 'pending':
            color = 'gray'
            prefix = "□"
        elif status == 'running':
            color = 'orange'
            prefix = "▶"
        elif status == 'success':
            color = 'green'
            prefix = "✔"
        elif status == 'error':
            color = 'red'
            prefix = "❌"
        else:
            color = 'gray'
            prefix = "□"

        time_spent = f" ({step_times[i]}秒)" if step_times[i] is not None else ""
        step_elements.append(html.Div(f"{prefix} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))

    # 決定進度文字
    if progress['error']:
        label = f"流程中斷：{step_names[current]} 發生錯誤"
        disabled = True
        to_comment_button = None  # 發生錯誤時不顯示按鈕
    elif current >= steps_total - 1:
        label = "完成所有步驟！"
        disabled = True
        to_comment_button = html.Button("報告完成，提供反饋", id="to-comment", n_clicks=0)  # 完成後顯示按鈕
    else:
        label = f"目前進度：{current + 1}/{steps_total}"
        disabled = False
        to_comment_button = None  # 尚未完成時，不顯示按鈕

    return step_elements, current + 1, label, chart, to_comment_button, disabled

if __name__ == "__main__":
    app.run(debug=True)
